In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import pytorch_lightning as pl
from PIL import Image
import numpy as np
from livecell_tracker.preprocess.utils import standard_preprocess, normalize_img_to_uint8

In [ ]:
from livecell_tracker.classification.cnn_mitosis_apoptosis import MitApopImageClassifier, MitApopImageDataset

In [ ]:
import matplotlib.pyplot as plt
img_dataset = MitApopImageDataset(dir_path="../datasets/wwk_train/A549_icnn_am_train/")
plt.imshow(np.array(img_dataset[0][0][0]))

In [4]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

classifier = MitApopImageClassifier(dir_path="../datasets/wwk_train/A549_icnn_am_train/", val_dir_path="../datasets/wwk_train/A549_icnn_am_val/")
logger = TensorBoardLogger(save_dir=".", name="./sc_classify_MA-no-resize")
last_models_checkpoint_callback = ModelCheckpoint(
    save_last=True,
    filename="{epoch}-{global_step}",
)
train_loss_callback = pl.callbacks.ModelCheckpoint(
        monitor="train_loss",
        dirpath="sc_classify_mitosis_apoptosis",
        filename="model-{epoch:02d}-{train_loss:.2f}",
        save_top_k=3,
        mode="min",
)
ckpt_callbacks = [last_models_checkpoint_callback, train_loss_callback]
trainer = pl.Trainer(
    gpus=1,
    max_epochs=10000,
    checkpoint_callback=ckpt_callbacks,
    logger=logger,
)
trainer.fit(classifier)
